In [90]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import xmltodict
import re

urls = ["https://hebbarskitchen.com/post-sitemap1.xml", "https://hebbarskitchen.com/post-sitemap2.xml"]
urldict = []

for url in urls:
    response = requests.get(url)
    xml = response.text
    soup = BeautifulSoup(xml)
    
    urlTags = soup.find_all("url")
    
    for urlTag in urlTags:
        urldict.append(urlTag.findNext("loc").text)
recipedict = []
for url in urldict:
    if re.match("^(https):\/\/[a-z]*\.(com)\/[a-z,A-Z,0-9,-]*\/$", url) != None:
        response = requests.get(url)
        soup = BeautifulSoup(response.text)
        if soup.find("span", class_="wprm-recipe-course") != None:
            recipedict.append([soup.find("h2", class_="wprm-recipe-name").text,url,soup.find("span", class_="wprm-recipe-course").text])
        
len(recipedict)
df = pd.DataFrame(recipedict, columns=["Name","URL","Course"])
df.to_json(r'HebbarsRecipes.json', orient='index')